In [1]:
from elasticsearch import Elasticsearch
import json

In [2]:
client = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme' : 'http'}])
client.ping()

True

In [3]:
indexName = "ticket"
if client.indices.exists(index=indexName):
    client.indices.delete(index=indexName)

In [4]:
ticket_settings = {
  "settings": {
    "analysis": {
      "filter": {
        "ru_stop": {
          "type": "stop",
          "stopwords": "_russian_"
        },
        "snow_ru_stemmer": {
          "type": "snowball",
          "language": "russian"
        }
      },
      "analyzer": {
        "custom_analyzer": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "ru_stop",
            "snow_ru_stemmer"
          ]
        }
      }
    }
  }
}

In [5]:
ticket_mappings = {
    "mappings": {
        "properties": {
            "user_id": {"type": "text"},
            "personal": {"type": "text", "analyzer": "custom_analyzer"},
            "date_purchase": {"type": "date"},
            "price": {"type": "float"},
            "train_id": {"type": "integer"}
        }
    }
}

In [6]:
client.indices.create(index=indexName, body={**ticket_settings, **ticket_mappings})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ticket'})

In [7]:
with open('../data/tickets.json', 'r') as f:
    trains = json.load(f)

for data in trains:
    try:
        client.index(
            index=data["index"],
            id=data["id"],
            body=data["body"]
        )
    except Exception as e:
        print(e, end="")

In [8]:
search_query = {
    "query": {
        "match_all": {}
    }
}
search_results = client.search(index=indexName, body=search_query)
hits = search_results["hits"]["hits"]
for hit in hits:
    print(hit["_source"])